In [ ]:
import logging
import json
import requests
import os
from typing import Any, Callable, Set
from azure.identity import AzureCliCredential  # or DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.keyvault.secrets import SecretClient


# [START enable_tracing]
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor


# --------------------------------------------------------------------
# 1) Load Environment Variables (Locally) or from Key Vault
# --------------------------------------------------------------------
# If you're storing secrets in .env locally, do:

keyvault_name = "keyvaultforbot"
kv_client = SecretClient(f"https://{keyvault_name}.vault.azure.net/", AzureCliCredential())

# Pull secrets from environment variables
try:
    PROJECT_CONNECTION_STRING = kv_client.get_secret("project-connection-string").value
    BSKY_USERNAME             = kv_client.get_secret("bskyusername").value
    BSKY_PASSWORD             = kv_client.get_secret("bskypw").value
    BING_API_KEY              = kv_client.get_secret("bing-search-api").value
except Exception as e:
    logging.error(f"Error retrieving secrets from Key Vault: {e}")
    raise

def retrieve_secret(secret_name: str, kv_client, suppress_logs: bool = False) -> str:
    """
    Retrieves a secret from Azure Key Vault, with an option to suppress logging.
    """

    try:
        secret_value = kv_client.get_secret(secret_name).value
        return secret_value
    except Exception as e:
        logging.error(f"Error retrieving secret '{secret_name}' from Key Vault: {e}")
        raise

# 1) Create the Azure AI Client from your connection string
project_client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),  # or DefaultAzureCredential()
        conn_str=PROJECT_CONNECTION_STRING
    )

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

# enable additional instrumentations
project_client.telemetry.enable()

scenario = os.path.basename("sample_agents_basics_with_azure_monitor_tracing")
tracer = trace.get_tracer("my_notebook_tracer")
        
# --------------------------------------------------------------------
# 4) Run Pipeline (mimics the Azure AI Projects logic)
# --------------------------------------------------------------------
def run_ai_project_pipeline():
    """
    Creates an Azure AI Projects agent, runs a conversation, and cleans up.
    """
    
    instructions = """
    You are a helpful assistant with the goal to give an overview of the latest relevant AI news
    """
    with tracer.start_as_current_span(scenario):
        with project_client:
            # 4) Create and run the agent
            agent = project_client.agents.create_agent(
                model="gpt-4o",
                name="my-agent",
                instructions=instructions,
                #toolset=toolset
            )
            

            # 5) Create a thread for conversation
            thread = project_client.agents.create_thread()

            # 6) Create a user message
            message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content="Hello! I'd like to have the latest AI news."
            )
            logging.info(f"Created message, ID: {message.id}")
            print(f"Message ID: {message.id}")

            # 7) Create and process a run
            run = project_client.agents.create_and_process_run(
                thread_id=thread.id,
                assistant_id=agent.id
            )
            logging.info(f"Run finished with status: {run.status}")
            print(f"Run Status: {run.status}")

            if run.status == "failed":
                logging.error(f"Run failed: {run.last_error}")

            # 8) List all messages (including the final assistant message)
            messages = project_client.agents.list_messages(thread_id=thread.id)
            logging.info("----- MESSAGES -----")
            for msg in messages["data"]:
                if msg["role"] == "assistant":
                    logging.info("Assistant says:")
                    for part in msg["content"]:
                        if part["type"] == "text":
                            logging.info(part["text"]["value"])
                            print(part["text"]["value"])
                elif msg["role"] == "system":
                    logging.info("System says:")
                    for part in msg["content"]:
                        if part["type"] == "text":
                            logging.info(part["text"]["value"])
                            print(part["text"]["value"])
                elif msg["role"] == "user":
                    logging.info(f"User says: {msg['content'][0]['text']['value']}")
                    print(f"User says: {msg['content'][0]['text']['value']}")
            # 9) Cleanup: delete the agent
            project_client.agents.delete_agent(agent.id)
            logging.info("Deleted agent successfully.")

# --------------------------------------------------------------------
# 5) Local Test Entry Point
# --------------------------------------------------------------------
# Simply call run_ai_project_pipeline() in your notebook to test locally.

run_ai_project_pipeline()

ResourceNotFoundError: Application Insights resource was not enabled for this Project.